In [1]:
import pandas as pd
from numble_data_preprocess import numble_preprocess, label_encoding
from collected_data_preprocess import nice_data_prepare, nice_combine, incruit_prepare, find_incruit_company
from financial_data_preprocess import make_financial_yeardata, delete_semi_active, delete_normal_active, delete_inactive, make_financial_column, replace_outlier, is_outlier

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False

# 1. 데이터 전처리
## 1) 비재무 데이터 전처리
### (1) 넘블 제공 데이터

In [2]:
# 데이터 가져오기
active_df = pd.read_excel("../data/1_Active_MS_Business_Info.xlsx", sheet_name = 0)
active_record_df = pd.read_excel("../data/1_Active_MS_Business_Info.xlsx", sheet_name = 2)
inactive_df = pd.read_excel("../data/2_Inactive_MS_Business_Info.xlsx", sheet_name = 0)
inactive_record_df = pd.read_excel("../data/2_Inactive_MS_Business_Info.xlsx", sheet_name = 1)

# 컬럼명 변경, 이상치 처리, 휴폐업 컬럼 추가, 결측치 대체
df_active_left, df_inactive_left = numble_preprocess(active_df, inactive_df, active_record_df, inactive_record_df)

### (2) 외부 데이터
### - NICE : nice 홈페이지 사업자번호 조회
- 1. 기업입지 (도심/부도심 - 기준: 구와 읍이 들어가는 행정구역을 주소로 가지면 도심
- 2. 산업등급


In [4]:
# 크롤링 한 데이터 csv파일 불러오기
nice_active_df = pd.read_excel("../data/외부data/act_company_information.xls", sheet_name = 0)
nice_inactive_df = pd.read_excel("../data/외부data/rest_company_information.xls", sheet_name = 0)

# 결측치 대체, 컬럼명 변경, 기업입지와 행정구역 컬럼 만들기
nice_active_df, nice_inactive_df = nice_data_prepare(nice_active_df, nice_inactive_df)

# 넘블 비재무데이터와 결합하고 결측치 대체
df_active, df_inactive = nice_combine(df_active_left, df_inactive_left, nice_active_df, nice_inactive_df)

### - Incruit 인크루트 (우수기업)
- 이노비즈, 메인비즈, 강소기업, 우수중소기업

In [7]:
# 인크루트 데이터파일 불러오기
incruit_df = pd.read_csv('../data/외부data/4_Incruit_Data.csv')

incruit_company = incruit_df['company_name'].tolist()
incruit_result = incruit_prepare(incruit_company)

# 우수기업에 해당하는 기업 찾기
find_incruit_company(incruit_result, df_active, df_inactive)

### (3) 비재무데이터 하나의 데이터프레임으로 결합
- 넘블데이터 + 외부 수집 비재무데이터

In [11]:
# 데이터 결합
nonfinancial_df = pd.concat([df_active, df_inactive])

# 인덱스 reset
nonfinancial_df.reset_index(drop=True, inplace=True)

# 기업명 컬럼 drop
nonfinancial_df.drop(['기업명'], axis=1, inplace=True)

# 라벨 인코딩
nonfinancial_df = label_encoding(nonfinancial_df)

### ** 스크래핑을 위한 데이터프레임 만들고 저장

In [25]:
#스크래피를 위한 데이터프레임 만들기
normal_active_company = df_active[['기업명']].loc[df_active['휴폐업이력']==0]
semi_active_company = df_active[['기업명','휴폐업발생일자']].loc[df_active['휴폐업이력']==1]
inactive_company = df_inactive[['기업명', '휴폐업발생일자']]

# 데이터프레임 csv로 저장
normal_active_company.to_csv('normal_active_company.csv', mode='w')
semi_active_company.to_csv('semi_active_company.csv', mode='w')
inactive_company.to_csv('inactive_company.csv', mode='w')

## 2) 재무데이터 전처리
- 휴폐업 기업은 휴폐업시점에서 직전 년도의 재무정보 사용
- 액티브 기업은 가장 최근 재무정보 사용

In [14]:
# 재무데이터 불러오기
financial_df = pd.read_csv("../data/3_Financial_Data.txt", sep = "\t", encoding = "cp949")

# 재무데이터 연 데이터로 결합
financial_df = make_financial_yeardata(financial_df)

# 재무데이터 처리를 위해 df_active를 액티브(휴폐업이력 有), 액티브(휴폐업이력 無), 현재 휴폐업 으로 나누기
df_semi_active = df_active.loc[df_active['휴폐업이력']==1]
df_normal_active = df_active.loc[df_active['휴폐업이력']==0]

# 휴폐업 발생일자, 휴폐업이력 새로운 컬럼으로 추가
for_merge_semi_active = df_semi_active[['사업자번호', '휴폐업발생일자','휴폐업이력']]
for_merge_normal_active = df_normal_active[['사업자번호', '휴폐업발생일자','휴폐업이력']]
for_merge_inactive = df_inactive[['사업자번호', '휴폐업발생일자','휴폐업이력']]

semi_active_fin_df = pd.merge(financial_df, for_merge_semi_active, left_on='사업자번호', right_on='사업자번호', how='inner')
normal_active_fin_df = pd.merge(financial_df, for_merge_normal_active, left_on='사업자번호', right_on='사업자번호', how='inner')
inactive_fin_df = pd.merge(financial_df, for_merge_inactive, left_on='사업자번호', right_on='사업자번호', how='inner')

# 안 쓸 재무데이터 삭제
semi_active_fin_df = delete_semi_active(semi_active_fin_df)
normal_active_fin_df = delete_normal_active(normal_active_fin_df)
inactive_fin_df = delete_inactive(inactive_fin_df)

# 재무데이터df 모두 결합
financial_df = pd.concat([semi_active_fin_df, normal_active_fin_df ,inactive_fin_df])

# 인덱스 reset
financial_df.reset_index(drop=True, inplace=True)

# 추가 컬럼 만들기 - 총자본순이익율, 이자보상비율, 이자비용, 금융비용부담률 , 매출액증가율, 순이익증가율 
preprocessed_financial_df = make_financial_column(financial_df)

# 휴폐업발생일자 str타입으로 변환
preprocessed_financial_df['휴폐업발생일자'] = preprocessed_financial_df['휴폐업발생일자'].astype('str')

### (1) 재무데이터 이상치 제거
- IQR 이용


In [22]:
# 숫자 표현 바꾸기
pd.options.display.float_format = '{:.2f}'.format

# 이상치 제거
preprocessed_financial_df = replace_outlier(preprocessed_financial_df)

In [32]:
# IQR 이용해서 이상치 제거하기 전 데이터
preprocessed_financial_df.describe()

,사업자번호,휴폐업이력,기업순이익률(%),유보액/총자산(%),유보액/납입자본(%),매출액총이익률(%),매출액영업이익률(%),매출액순이익률(%),수지비율(%),경상수지비율,...,부채총계대 매출액(%),총자본회전율(회),재고자산회전율(회),매출채권회전율(회),매입채무회전율(회),총자본순이익률,이자보상비율,금융비용부담률(%),매출액증가율(%),순이익증가율(%)
count,27567.00,27567.00,25306.00,27176.00,27033.00,25984.00,25989.00,25984.00,26905.00,18014.00,...,25812.00,25295.00,18700.00,21175.00,18003.00,27205.00,27206.00,27202.00,27538.00,27559.00
mean,3260511996.02,0.08,3.49,-40.08,-80837.11,10.28,-235.80,-461.72,519.84,-56583495.79,...,837.75,1.23,167.22,61.69,149.39,-12.37,-6.90,-9.47,166.15,21.88
std,2174036168.78,0.27,22.49,8541.09,9905608.67,1350.92,7241.15,17269.91,4269.20,578134922.80,...,4660.02,2.64,2052.29,884.62,1911.71,2710.82,1208.71,4884.59,2251.43,5312.11
min,1018100340.00,0.00,-1539.47,-1401856.01,-1581697000.00,-123719.24,-707996.32,-1657346.39,-157.56,-52994789497.00,...,0.00,0.00,0.00,0.00,0.00,-422329.66,-199250.00,-796085.84,-125337.94,-478925.00
25%,1308159277.00,0.00,0.57,0.98,12.10,10.91,-0.42,-1.46,90.34,-8673412.00,...,31.69,0.28,3.82,5.01,7.73,0.65,0.49,0.00,2.94,-65.46
50%,2208744637.00,0.00,3.64,21.92,538.61,21.46,3.61,2.61,97.17,-538227.30,...,69.74,0.81,9.15,8.05,14.10,8.56,0.92,7.19,14.63,2.85
75%,5138110847.00,0.00,8.58,47.13,2176.95,46.10,10.84,8.65,102.56,152.90,...,206.87,1.46,26.16,14.54,29.84,23.17,1.13,22.25,39.73,83.36
max,8998800785.00,1.00,428.72,19996.30,99958.11,604.03,1556.88,92467.80,99360.73,99360.73,...,96253.02,234.60,98402.33,56084.63,95912.15,26885.24,1328.58,79119.84,96238.15,97772.25


In [37]:
fin_column_list = preprocessed_financial_df.columns.tolist()[4:]

# IQR 구하기 위한 dict 만들기

dict_q3 = {}
dict_q1 = {}
dict_iqr = {}

for col in fin_column_list:
    
    q3 = preprocessed_financial_df[col].quantile(0.75) 
    q1 = preprocessed_financial_df[col].quantile(0.25)
    iqr = q3 - q1

    dict_q3[col]=q3
    dict_q1[col]=q1
    dict_iqr[col]=iqr

# 이상치제거
for col in fin_column_list:
    preprocessed_financial_df[col] = preprocessed_financial_df.apply(is_outlier, column=col, dict_q3 = dict_q3, dict_q1=dict_q1, dict_iqr=dict_iqr, axis=1)

In [52]:
# IQR 이용해서 이상치 제거 후
preprocessed_financial_df.describe()

,사업자번호,휴폐업이력,기업순이익률(%),유보액/총자산(%),유보액/납입자본(%),매출액총이익률(%),매출액영업이익률(%),매출액순이익률(%),수지비율(%),경상수지비율,...,부채총계대 매출액(%),총자본회전율(회),재고자산회전율(회),매출채권회전율(회),매입채무회전율(회),총자본순이익률,이자보상비율,금융비용부담률(%),매출액증가율(%),순이익증가율(%)
count,27567.00,27567.00,22000.00,26280.00,22761.00,22328.00,20482.00,20230.00,21078.00,14873.00,...,21398.00,23682.00,15950.00,18388.00,15653.00,22255.00,24830.00,23848.00,24411.00,23140.00
mean,3260511996.02,0.08,4.39,25.77,827.14,24.10,4.93,3.94,95.59,-2223913.41,...,84.11,0.88,11.51,8.55,16.01,10.20,0.81,9.89,18.83,4.72
std,2174036168.78,0.27,5.90,31.60,1366.54,21.25,8.06,7.20,8.57,4304342.70,...,88.04,0.74,12.28,5.66,12.83,15.88,0.47,15.12,21.23,103.33
min,1018100340.00,0.00,-11.43,-68.10,-3229.41,-41.80,-17.30,-16.63,72.02,-21679841.60,...,0.00,0.00,0.00,0.00,0.00,-33.13,-0.46,-33.35,-9.20,-288.51
25%,1308159277.00,0.00,1.21,2.81,7.08,10.03,0.90,0.61,91.37,-2221402.74,...,26.49,0.24,3.20,4.66,7.01,1.55,0.56,0.00,2.03,-53.42
50%,2208744637.00,0.00,3.64,23.10,404.78,18.50,3.79,2.99,96.63,-81260.58,...,54.72,0.75,7.19,7.11,12.14,8.12,0.92,6.24,11.34,0.00
75%,5138110847.00,0.00,7.66,47.83,1368.23,32.73,8.85,7.51,99.38,248.24,...,103.19,1.30,15.15,11.04,21.01,18.21,1.10,17.79,28.18,52.90
max,8998800785.00,1.00,20.57,116.28,5423.67,98.84,27.71,23.82,120.85,99360.73,...,469.54,3.23,59.63,28.82,62.93,56.93,2.07,55.62,94.88,306.41


### 2) MICE 이용하여 결측치 처리

In [54]:
preprocessed_financial_df

,사업자번호,결산연도,휴폐업발생일자,휴폐업이력,기업순이익률(%),유보액/총자산(%),유보액/납입자본(%),매출액총이익률(%),매출액영업이익률(%),매출액순이익률(%),...,부채총계대 매출액(%),총자본회전율(회),재고자산회전율(회),매출채권회전율(회),매입채무회전율(회),총자본순이익률,이자보상비율,금융비용부담률(%),매출액증가율(%),순이익증가율(%)
0,1078600246,2020,20210831.0,1.00,16.01,29.55,996.36,20.26,12.16,7.05,...,39.86,2.17,NaN,13.22,7.92,36.75,1.30,10.66,48.65,90.28
1,1078784490,2021,20220616.0,1.00,NaN,NaN,-1331.06,NaN,-11.50,NaN,...,123.90,2.98,NaN,14.06,NaN,26.94,0.76,-6.60,40.69,-9.41
2,1208776635,2021,20220602.0,1.00,3.33,18.51,161.46,28.93,2.51,1.65,...,54.45,1.23,3.49,2.16,5.19,7.08,0.98,38.86,6.99,NaN
3,1288629293,2019,20200612.0,1.00,-2.46,NaN,-873.19,NaN,-16.16,NaN,...,NaN,0.18,12.16,26.43,NaN,12.65,1.20,-7.43,19.59,-43.85
4,1358637850,2020,20210305.0,1.00,NaN,20.18,784.86,NaN,-2.39,-2.45,...,3.83,NaN,NaN,NaN,NaN,NaN,1.34,NaN,85.58,-116.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27562,8988800759,2021,99999999.0,0.00,-6.89,4.08,439.00,15.93,2.10,-6.48,...,67.97,1.35,11.30,6.15,24.03,NaN,-0.43,-20.55,5.34,-144.58
27563,8998600665,2021,99999999.0,0.00,2.85,1.10,19.53,26.41,17.12,5.18,...,NaN,0.17,NaN,22.93,9.19,13.31,1.05,6.11,NaN,63.13
27564,8998800150,2021,99999999.0,0.00,3.99,47.80,1304.71,NaN,3.62,3.59,...,51.07,0.91,NaN,23.95,8.08,6.63,0.87,24.02,2.80,NaN
27565,8998800427,2021,99999999.0,0.00,NaN,4.10,150.90,7.12,-6.15,-8.96,...,70.09,1.48,6.79,7.95,9.79,NaN,0.72,-11.68,33.40,-123.73


In [55]:
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

missing_data_df = preprocessed_financial_df.drop(['사업자번호', '결산연도', '휴폐업발생일자'], axis=1)

# 마지막에 사업자 번호 붙여주기 위해서 df 생성
company_num_df = preprocessed_financial_df['사업자번호']

In [57]:
# MICE 이용하여 결측치 대체
imputer_mice = IterativeImputer(max_iter=30,random_state=83) # default : max_iter =10 
numeric_data = imputer_mice.fit_transform(missing_data_df)

missing_columns = list(missing_data_df.columns)
row_indices = [x for x in range(27567)]
data_df = pd.DataFrame(numeric_data, index=row_indices, columns=missing_columns)

# 사업자번호 붙여주기
final_financial_df = pd.concat([company_num_df, data_df], axis=1)

c:\Users\bin\anaconda3\envs\numble\lib\site-packages\sklearn\impute\_iterative.py:713: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [62]:
final_financial_df.describe()

,사업자번호,휴폐업이력,기업순이익률(%),유보액/총자산(%),유보액/납입자본(%),매출액총이익률(%),매출액영업이익률(%),매출액순이익률(%),수지비율(%),경상수지비율,...,부채총계대 매출액(%),총자본회전율(회),재고자산회전율(회),매출채권회전율(회),매입채무회전율(회),총자본순이익률,이자보상비율,금융비용부담률(%),매출액증가율(%),순이익증가율(%)
count,27567.00,27567.00,27567.00,27567.00,27567.00,27567.00,27567.00,27567.00,27567.00,27567.00,...,27567.00,27567.00,27567.00,27567.00,27567.00,27567.00,27567.00,27567.00,27567.00,27567.00
mean,3260511996.02,0.08,4.65,25.03,937.40,25.72,4.94,3.35,95.98,-2311183.57,...,120.24,0.92,11.49,8.49,16.69,10.21,0.81,10.66,18.81,7.35
std,2174036168.78,0.27,6.24,31.55,1351.26,20.17,6.95,8.80,9.32,3282109.64,...,110.26,0.78,10.23,4.96,10.94,16.04,0.47,14.54,20.11,96.03
min,1018100340.00,0.00,-17.72,-68.10,-3229.41,-41.80,-17.30,-59.01,21.33,-21679841.60,...,-44.23,-1.18,-13.75,-2.25,-25.41,-40.03,-3.04,-33.35,-9.20,-288.51
25%,1308159277.00,0.00,1.18,2.24,27.69,11.59,1.89,0.12,91.54,-3205943.08,...,33.69,0.26,4.34,5.18,8.99,1.32,0.59,0.00,2.96,-34.40
50%,2208744637.00,0.00,3.80,22.80,567.06,22.14,4.91,2.72,96.88,-1575661.37,...,76.38,0.79,9.55,7.64,15.22,8.20,0.90,8.05,13.34,4.92
75%,5138110847.00,0.00,8.15,46.73,1736.02,34.63,6.49,7.54,99.76,37.96,...,196.06,1.38,15.49,10.60,21.80,18.37,1.09,18.96,26.51,50.84
max,8998800785.00,1.00,37.20,116.28,5423.67,126.70,27.71,77.64,157.84,3757019.97,...,615.17,4.07,59.63,28.82,62.93,71.50,5.18,55.62,94.88,306.41


## 3) 최종 데이터 결합
- 비재무 데이터프레임과 재무 데이터프레임 결합
- nonfinancial_df + final_financial_df

In [206]:
final_data = pd.merge(nonfinancial_df, final_financial_df, left_on='사업자번호', right_on='사업자번호', how='left')
final_data.drop(['사업자번호','휴폐업발생일자', '휴폐업이력_y'], axis=1, inplace=True)
final_data.rename(columns={'휴폐업이력_x':'휴폐업이력'}, inplace=True)

# 재무데이터 결측치 처리 0
final_data.fillna(0, inplace=True)

In [207]:
# csv파일로 저장
final_data.to_csv('../data/final_data.csv')